In [1]:
import openai
from openai import OpenAI

import networkx
from networkx.algorithms.community import k_clique_communities
import networkx.algorithms.approximation as approximation

import numpy
from scipy import stats
import pandas

import matplotlib
import matplotlib.pyplot
import matplotlib.animation as animation
import matplotlib.patches as patches
import seaborn

import os.path
import datetime
import re
import json
import random

In [2]:
data_path = "national_M2023_dl.xlsx"
raw_data = pandas.read_excel(data_path, header=0)

In [3]:
raw_data.describe()

,AREA,AREA_TYPE,NAICS,OWN_CODE,TOT_EMP,EMP_PRSE,JOBS_1000,LOC_QUOTIENT,PCT_TOTAL,PCT_RPT,MEAN_PRSE
count,1403.0,1403.0,1403.0,1403.0,1.403000e+03,1403.000000,0.0,0.0,0.0,0.0,1403.000000
mean,99.0,1.0,0.0,1235.0,5.411738e+05,2.728011,NaN,NaN,NaN,NaN,1.061083
std,0.0,0.0,0.0,0.0,4.241159e+06,3.301950,NaN,NaN,NaN,NaN,1.325519
min,99.0,1.0,0.0,1235.0,2.600000e+02,0.000000,NaN,NaN,NaN,NaN,0.000000
25%,99.0,1.0,0.0,1235.0,2.038500e+04,0.900000,NaN,NaN,NaN,NaN,0.400000
50%,99.0,1.0,0.0,1235.0,7.223000e+04,1.700000,NaN,NaN,NaN,NaN,0.700000
75%,99.0,1.0,0.0,1235.0,2.755500e+05,3.350000,NaN,NaN,NaN,NaN,1.200000
max,99.0,1.0,0.0,1235.0,1.518539e+08,33.400000,NaN,NaN,NaN,NaN,14.600000


In [4]:
raw_data.head()

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,23.11,37.01,58.4,29050,35660,48060,76980,121470,NaN,NaN
1,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,56.19,81.29,111.36,54550,78330,116880,169090,231620,NaN,NaN
2,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-1000,Top Executives,...,49.74,79.57,#,46400,66170,103460,165500,#,NaN,NaN
3,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-1010,Chief Executives,...,99.37,#,#,80000,130840,206680,#,#,NaN,NaN
4,99,U.S.,1,US,0,Cross-industry,cross-industry,1235,11-1011,Chief Executives,...,99.37,#,#,80000,130840,206680,#,#,NaN,NaN


In [5]:
raw_data.drop(['AREA', 'AREA_TITLE', 'AREA_TYPE', 'PRIM_STATE', 'NAICS', 'OWN_CODE', 'ANNUAL', 'HOURLY', 'JOBS_1000', 'LOC_QUOTIENT'], axis=1, inplace=True)

In [6]:
parsed_data_folder = 'parsed-data'
gpt_key = ""
with open("gpt-key", 'r') as file:
    gpt_key = file.read()
openai_assistant = "asst_qfbffm1klm8Ozr7tqAJ26wkM"

In [7]:
client = OpenAI(api_key=gpt_key)
thread = client.beta.threads.create()
assistant_content = "You're a helpful research assistant. Your answers are short and strictly follow instruction. For each job title received, you provide a list with technical skills associated with the job, then a second list with psychological traits related to this job and finally a third list with certificates helpful to get this job. You provide only lists, nothing more."
print(thread)

Thread(id='thread_RHaAlEnKeFFvfdKACqo3lukZ', created_at=1725971507, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [8]:
def extract_lists(input_string):
    tech_skills = re.search(r'\*\*Technical Skills:\*\*(.*)\*\*Psychological Traits:\*\*', input_string, re.DOTALL)
    psych_traits = re.search(r'\*\*Psychological Traits:\*\*(.*)\*\*Certificates:\*\*', input_string, re.DOTALL)
    certificates = re.search(r'\*\*Certificates:\*\*(.*)', input_string, re.DOTALL)
    
    tech_skills = tech_skills.group(1) if tech_skills else ''
    psych_traits = psych_traits.group(1) if psych_traits else ''
    certificates = certificates.group(1) if certificates else ''
    
    def clean_and_split(text):
        cleaned_items = re.sub(r'\d+\.\s*', '', text).strip()
        return [item.strip() for item in cleaned_items.split('\n') if item.strip()]

    tech_list = clean_and_split(tech_skills) if tech_skills else []
    psych_list = clean_and_split(psych_traits) if psych_traits else []
    cert_list = clean_and_split(certificates) if certificates else []

    return tech_list, psych_list, cert_list

In [9]:
summarise_data_flag = False # to control for accidental triggering

for i, row in raw_data.iterrows():
    if '0000' not in row['OCC_CODE'] and summarise_data_flag:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": assistant_content},
                {"role": "user", "content": row['OCC_TITLE']}
            ]
        )
        response_message = response.choices[0].message.content.strip()
        try:
            tech_list, psych_list, cert_list = extract_lists(response_message)
        except Exception as e:
            tech_list = []
            psych_list = [] 
            cert_list = []
        
        temp_dict = {"code": row['OCC_CODE'],
                     "title": row['OCC_TITLE'],
                     "count": row['TOT_EMP'],
                     "tech-skills": tech_list,
                     "psych-skill": psych_list,
                     "certs": cert_list,
                     "hourly-rate": row['H_MEAN'],
                     "annual-rate": row['A_MEAN'],
                     "annual-percentile-10": row['A_PCT10'],
                     "annual-percentile-25": row['A_PCT25'],
                     "annual-percentile-50": row['A_MEDIAN'],
                     "annual-percentile-75": row['A_PCT75'],
                     "annual-percentile-90": row['A_PCT90']
                    }          
        out_file = open(f"{parsed_data_folder}/{temp_dict['code']}.json", "w")
        json.dump(temp_dict, out_file)
        out_file.close()

In [10]:
files = os.listdir(parsed_data_folder)
json_files = [file for file in files if file.endswith('.json')]
json_file_count = len(json_files)

print(f"Number of JSON files: {json_file_count}, number of rows in source data: {len(raw_data.index)} - the difference is because of ignoring aggregated jobs entries i.e. 'all occupations' or 'managers'")

random_json_file = random.choice(json_files)
random_json_file_path = os.path.join(parsed_data_folder, random_json_file)
with open(random_json_file_path, 'r') as f:
    json_data = json.load(f)

print(f"Randomly selected JSON file: {random_json_file}")
print(json.dumps(json_data, indent=4))

Number of JSON files: 1373, number of rows in source data: 1403 - the difference is because of ignoring aggregated jobs entries i.e. 'all occupations' or 'managers'
Randomly selected JSON file: 49-3052.json
{
    "code": "49-3052",
    "title": "Motorcycle Mechanics",
    "count": 14330,
    "tech-skills": [
        "Engine repair and maintenance",
        "Electrical systems diagnosis",
        "Transmission servicing",
        "Brake system repair",
        "Tire fitting and balance",
        "Fuel system maintenance",
        "Carburetor tuning",
        "Welding and fabrication"
    ],
    "psych-skill": [
        "Attention to detail",
        "Problem-solving skills",
        "Patience",
        "Mechanical aptitude",
        "Customer service orientation",
        "Adaptability",
        "Manual dexterity",
        "Teamwork"
    ],
    "certs": [
        "Motorcycle Mechanic Certification (e.g., from the National Institute for Automotive Service Excellence - ASE)",
        "Cer

In [11]:
tech_skills = {}
psych_skills = {}
certs = {}
for file in json_files:
    json_file_path = os.path.join(parsed_data_folder, file)
    json_data = None
    with open(json_file_path, 'r') as f:
        json_data = json.load(f)
        for tech_skill in json_data['tech-skills']:
            try:
                tech_skills[tech_skill]['jobs'].append(json_data['title'])
                tech_skills[tech_skill]['counts'].append(json_data['count'])
                tech_skills[tech_skill]['annual-percentile-10'].append(json_data['annual-percentile-10'])
                tech_skills[tech_skill]['annual-percentile-25'].append(json_data['annual-percentile-25'])
                tech_skills[tech_skill]['annual-percentile-50'].append(json_data['annual-percentile-50'])
                tech_skills[tech_skill]['annual-percentile-75'].append(json_data['annual-percentile-75'])
                tech_skills[tech_skill]['annual-percentile-90'].append(json_data['annual-percentile-90'])   
            except KeyError as e:
                tech_skills[tech_skill] = {'jobs': [json_data['title']],
                                           'counts': [json_data['count']],
                                           'annual-percentile-10': [json_data['annual-percentile-10']],
                                           'annual-percentile-25': [json_data['annual-percentile-25']],
                                           'annual-percentile-50': [json_data['annual-percentile-50']],
                                           'annual-percentile-75': [json_data['annual-percentile-75']],
                                           'annual-percentile-90': [json_data['annual-percentile-90']]}
        for psych_skill in json_data['psych-skill']:
            try:
                psych_skills[psych_skill]['jobs'].append(json_data['title'])
                psych_skills[psych_skill]['counts'].append(json_data['count'])
                psych_skills[psych_skill]['annual-percentile-10'].append(json_data['annual-percentile-10'])
                psych_skills[psych_skill]['annual-percentile-25'].append(json_data['annual-percentile-25'])
                psych_skills[psych_skill]['annual-percentile-50'].append(json_data['annual-percentile-50'])
                psych_skills[psych_skill]['annual-percentile-75'].append(json_data['annual-percentile-75'])
                psych_skills[psych_skill]['annual-percentile-90'].append(json_data['annual-percentile-90'])   
            except KeyError as e:
                psych_skills[psych_skill] = {'jobs': [json_data['title']],
                                           'counts': [json_data['count']],
                                           'annual-percentile-10': [json_data['annual-percentile-10']],
                                           'annual-percentile-25': [json_data['annual-percentile-25']],
                                           'annual-percentile-50': [json_data['annual-percentile-50']],
                                           'annual-percentile-75': [json_data['annual-percentile-75']],
                                           'annual-percentile-90': [json_data['annual-percentile-90']]}
        for cert in json_data['certs']:
            try:
                certs[cert]['jobs'].append(json_data['title'])
                certs[cert]['counts'].append(json_data['count'])
                certs[cert]['annual-percentile-10'].append(json_data['annual-percentile-10'])
                certs[cert]['annual-percentile-25'].append(json_data['annual-percentile-25'])
                certs[cert]['annual-percentile-50'].append(json_data['annual-percentile-50'])
                certs[cert]['annual-percentile-75'].append(json_data['annual-percentile-75'])
                certs[cert]['annual-percentile-90'].append(json_data['annual-percentile-90'])   
            except KeyError as e:
                certs[cert] = {'jobs': [json_data['title']],
                                           'counts': [json_data['count']],
                                           'annual-percentile-10': [json_data['annual-percentile-10']],
                                           'annual-percentile-25': [json_data['annual-percentile-25']],
                                           'annual-percentile-50': [json_data['annual-percentile-50']],
                                           'annual-percentile-75': [json_data['annual-percentile-75']],
                                           'annual-percentile-90': [json_data['annual-percentile-90']]}

out_file = open(f"tech_skills.json", "w")
json.dump(tech_skills, out_file)
out_file.close()

out_file = open(f"psych_skills.json", "w")
json.dump(psych_skills, out_file)
out_file.close()  

out_file = open(f"certs.json", "w")
json.dump(certs, out_file)
out_file.close()

In [12]:
print(f"Given {json_file_count} occupations, there're {len(tech_skills.keys())} technical skills, {len(psych_skills.keys())} psychologicall traits and {len(certs.keys())} certificates identified in total.")

Given 1373 occupations, there're 6026 technical skills, 973 psychologicall traits and 5126 certificates identified in total.
